## ECE495 Final Project: SNN and DNN on heart failure classification

Description here

#### Set up

**Ensure you are using your [495 Virtual Environment](https://github.com/kaitlin-fair/495venv_setup) before you begin!**  
    
Import Nengo and other supporting libraries into your program to get started:

In [1]:
import matplotlib.pyplot as plt
import numpy as np

import nengo
from nengo.processes import Piecewise
from nengo.processes import WhiteSignal
from nengo.solvers import LstsqL2

### Datasets

Links and description

**load them beloww**


### Preprocessing
Normalize each feature and flatten imported files to [instances, features]

### SNN
Using SNN to classify datasets

### DNN
Might have to switch to 487 folder

Using 8 layers and designated neuron numbers

In [2]:
print(path)

NameError: name 'path' is not defined